In [ ]:
from ipywidgets import interact, FloatSlider, SelectionSlider

plt.close('all'); fig, ax = plt.subplots(5, 2, figsize=(8, 9))
n = np.arange(0, 100, step=1); 
f = sfft.fftshift(sfft.fftfreq(n=len(n), d=1))
np.random.seed(0); C = 5*np.exp(-0.5*(n[:, np.newaxis] - n[:, np.newaxis].T)**2/10**2)
x_clean = np.random.multivariate_normal(np.zeros_like(n), C)
x_clean -= np.mean(x_clean)
x = x_clean + 2*np.random.randn(len(n))
def update(fc, L, window='rect'):
    [axx.cla() for axx in ax.ravel()]
    # Señal limpia
    ax[0, 0].plot(n, x_clean); 
    ax[0, 1].plot(f, sfft.fftshift(np.absolute(sfft.fft(x_clean))))
    ax[0, 0].set_title("Tiempo"); ax[0, 1].set_title("Frecuencia")    
    # Señal contaminada
    ax[1, 0].plot(n, x); ylims = ax[1, 0].get_ylim(); ax[0, 0].set_ylim(ylims)
    X = sfft.fft(x); 
    ax[1, 1].plot(f, sfft.fftshift(np.absolute(X)))
    # Filtro ideal
    kc = int(len(n)*fc)
    Hd = np.zeros_like(n, dtype=np.float); Hd[:kc] = 1.; Hd[len(Hd)-kc+1:] = 1.
    ax[2, 1].plot(f, sfft.fftshift(Hd))
    hd = np.real(sfft.ifftshift(sfft.ifft(Hd)))
    ax[2, 0].plot(n, hd)
    # Filtro real
    w = np.zeros_like(n, dtype=np.float); 
    if window == 'rect':
        w[len(w)//2-L//2:len(w)//2+L//2+1] = 1.;
    elif window == 'hann':
        w[len(w)//2-L//2:len(w)//2+L//2+1] = scipy.signal.hann(L+1)
    h = w*hd; H = sfft.fft(h); 
    print("NMSE entre H y Hd %0.4f" % (np.mean((np.absolute(H) - Hd)**2)/np.mean(Hd**2)))
    ax[3, 0].plot(n, h);  ylims2 = ax[3, 0].get_ylim(); 
    ax[3, 0].plot(n, w, '--'); ax[3, 0].set_ylim(ylims2)
    ax[3, 1].plot(f, sfft.fftshift(np.absolute(H)))
    # Señal filtrada
    xf = scipy.signal.convolve(x, h, mode='same')
    ax[4, 0].plot(n, np.real(xf)); 
    ax[4, 1].plot(f, sfft.fftshift(np.absolute(sfft.fft(xf)))); ax[4, 0].set_ylim(ylims)
    
interact(update, fc=FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1),
         L=SelectionSlider(options=[4, 6, 10, 20, 40, 60, 80], value=60),
         window=SelectionSlider(options=['rect', 'hann']));

In [ ]:
import librosa
data, sample_rate = librosa.load("../../data/nomekop.ogg", sr=44100)

fig, ax = plt.subplots(1, figsize=(8, 3))
freq, ttime, Sxx = scipy.signal.spectrogram(data, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax.pcolormesh(ttime, freq, 20*np.log10(Sxx+1e-4), cmap=plt.cm.magma); ax.set_ylim([0.0, 1e+4]);

Audio(data, rate=int(sample_rate), normalize=False)

In [ ]:
h = scipy.signal.firwin(numtaps=1000+1, cutoff=[200, 500], 
                        pass_zero=False, window='hann',
                        fs=sample_rate)

fig = plt.figure(figsize=(8, 6), tight_layout=True)
ax1 = plt.subplot2grid((2, 2 ), (0, 0)); ax1.set_title("Respuesta al impulso")
ax2 = plt.subplot2grid((2, 2 ), (0, 1)); ax2.set_title("Respuesta en frecuencia")
ax3 = plt.subplot2grid((2, 2 ), (1, 0), colspan=2, rowspan=1)

freq, response = scipy.signal.freqz(h)
ax1.plot(h); ax2.semilogy(0.5*sample_rate*freq/np.pi, np.abs(response));
data_filt = scipy.signal.convolve(data, h)
freq, ttime, Sxx = scipy.signal.spectrogram(data_filt, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax3.pcolormesh(ttime, freq, 20*np.log10(Sxx+1e-4), cmap=plt.cm.magma); ax3.set_ylim([0.0, 1e+4]);

Audio(data_filt, rate=int(sample_rate), normalize=False)